### Eventloop (Lösung)

In [12]:
import threading
from ipywidgets import Output, Button
from ipycanvas import Canvas
from IPython.display import display


layout = {'border': '1px solid black'}
out = Output(layout=layout)
canvas = Canvas(width=100, height=100, layout=layout)
button_stop = Button(description='Stop')
button_start = Button(description='Start')

state = {'pos': (50, 50)}
key_direction = {'ArrowUp': (0, -5),
                 'ArrowDown': (0, 5),
                 'ArrowLeft': (-5, 0),
                 'ArrowRight': (5, 0),
                 }

stop_event = threading.Event()
event_queue = []


@out.capture(clear_output=True)
def on_key_down(key, *flags):
    print(key)
    if key in key_direction:
        direction = key_direction[key]
        event_queue.insert(0, direction)


def move(dx, dy):
    x0, y0 = state['pos']
    x, y = (x0+dx, y0+dy)

    canvas.stroke_lines([(x0, y0), (x, y)])
    state['pos'] = (x, y)

    if x < 0 or y < 0 or x > 100 or y > 100:
        canvas.fill_rect(0, 0, canvas.width, canvas.height)
        stop_event.set()


@out.capture()
def start():
    canvas.clear()
    stop_event.clear()
    event_queue.clear()
    state['pos'] = (50, 50)
    event_queue.insert(0, (5, 0))
    event_loop()
    canvas.focus()


def event_loop(last_event=None, count=0):
    if event_queue:
        direction = event_queue.pop()
    else:
        direction = last_event

    dx, dy = direction
    move(dx, dy)

    if not stop_event.is_set():
        thread = threading.Timer(1, event_loop, args=(direction, count+1))
        thread.name = f'Eventloop-{count}'
        thread.start()


canvas.on_key_down(on_key_down)
button_stop.on_click(lambda bt: stop_event.set())
button_start.on_click(lambda bt: start())

display(canvas, button_stop, button_start, out)
canvas.focus()  # Canvas-Widget hoert auf Events, ohne das es zuerst angeklickt werden muss

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

Button(description='Stop', style=ButtonStyle())

Button(description='Start', style=ButtonStyle())

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…